In [1]:
import nbimporter
import datetime
import csv
import Gdata
from Gdata import Gdata
import Weather
from Weather import Weather, enmWeatherComponent
import WindowMng
from WindowMng import WindowMng, Window
import WallMng
from WallMng import WallMng
from Wall import Layer, Wall
import Schedule
from Schedule import Schedule

Importing Jupyter notebook from Gdata.ipynb
Importing Jupyter notebook from Weather.ipynb
Importing Jupyter notebook from SolarPosision.ipynb
Importing Jupyter notebook from WindowMng.ipynb
Importing Jupyter notebook from WallMng.ipynb
Importing Jupyter notebook from Wall.ipynb
Importing Jupyter notebook from Schedule.ipynb
Importing Jupyter notebook from ACSet.ipynb
Importing Jupyter notebook from AnnualCal.ipynb
Importing Jupyter notebook from mdlLibrary.ipynb
Importing Jupyter notebook from Appl.ipynb
Importing Jupyter notebook from Light.ipynb
Importing Jupyter notebook from LocalVent.ipynb
Importing Jupyter notebook from Resi.ipynb


# 室温・熱負荷計算のメイン関数

In [2]:
class heat_load_main():
    #シミュレーション全体の設定条件の読み込み
    def Gdata_init(self):
        self.__objGdata = Gdata(3600,20,1,1,12,31,50,True,"面積比",0.5,False)
        return self.__objGdata
    
    def Weather_init(self):
        #print('Weather_init')
        self.__Weather = Weather(34.6583333333333, 133.918333333333, 135)
        return self.__Weather
    
    #スケジュールの初期化
    def Schedule_init(self):
        self.__sch = Schedule()
        return self.__sch
        #print(self.__sch.ACSet('主たる居室','', '温度', datetime.datetime.strptime('2017/06/01 12:06', '%Y/%m/%d %H:%M')))  
    
    #開口部の登録
    def window_mng(self):
        with open('開口部.csv') as f:
            reader = csv.reader(f)
            header = next(reader)
            
            windowlist = []
            for row in reader:
                windowname = row[0]                    #開口部名称
                shgc = float(row[1])                   #日射熱取得率
                tau = float(row[2])                    #日射透過率
                B = float(row[3])                      #吸収日射取得率
                Uw = float(row[4])                     #開口部熱貫流率
                hic = float(row[5])                     #室内側対流熱伝達率
                hir = float(row[6])                     #室内側放射熱伝達率
                ho = float(row[7])                     #室外側総合熱伝達率
                Eo = float(row[8])                     #室外側放射率
                
                windowspec = {
                    'Name': windowname, 
                    'Eta': shgc, 
                    'SolarTrans': tau,
                    'SolarAbsorp': B,
                    'Uw': Uw,
                    'OutHeatTrans'   : ho,
                    'OutEmissiv'     : Eo,
                    'InConHeatTrans' : hic,
                    'InRadHeatTrans' : hir
                }
                
                windowlist.append(windowspec)
            
            #開口部クラスへの登録
            d = { 'Windows': windowlist }
            #print(d)
            self.__windwo_mng = WindowMng(d)
            
            return self.__windwo_mng
    
    #壁体構成の登録
    def wall_mng(self):
        #self.__wall_mng = WallMng()
        
        surf_heat_trans = []
        #表面熱伝達率の読み込み
        with open('表面熱伝達率.csv') as f:
            reader = csv.reader(f)
            header = next(reader)
            for row in reader:
                partname = row[0]                     #部位名称
                hi = float(row[1])                           #室内側総合熱伝達率
                hir = 5.0                             #室内側放射熱伝達率
                hic = hi - hir                        #室内側対流熱伝達率
                ho = float(row[2])                           #室外側総合熱伝達率
                dct = {'partname':partname, 'hi':hi, 'hir':hir, 'hic':hic, 'ho':ho}
                surf_heat_trans.append(dct)
            
        #壁体構成の読み込み
        with open('壁体構成.csv') as f:
            reader = csv.reader(f)
            header = next(reader)
            
            Walls = []
            layers = []                               #層構成収録リスト
            prev_partname = ''
            for row in reader:
                partname = row[0]                     #部位名称
                #print(partname)
                #部位名称が変更されていたら新しい部位とみなす
                if partname != prev_partname and len(layers) > 0:
                    #表面熱伝達率の取得
                    for surfstrudct in surf_heat_trans:
                        if surfstrudct['partname'] == partname:
                            wallstruct = {'Name':prev_partname, \
                                          'OutEmissiv':0.90, \
                                          'OutSolarAbs':0.80, \
                                          'InConHeatTrans':surfstrudct['hic'], \
                                          'InRadHeatTrans':surfstrudct['hir'], \
                                          'Layers':layers}
                            Walls.append(wallstruct)
                            wallstruct.clear
                            layers.clear
                
                name = row[1]                         #部材名称
                Lam = float(row[2])                          #熱伝導率
                if len(row[3]) > 0:
                    Spcheat = float(row[3])                      #容積比熱
                else:
                    Spcheat = 0.0

                if len(row[4]) > 0:
                    Dim = float(row[4]) / 1000.                  #厚さ
                else:
                    Dim = 0.0
                #層構成の追加
                layers.append({'Name': name, 'Cond': Lam, 'Thick': Dim, 'SpecH': Spcheat})
                prev_partname = partname

            #バッファに残った壁体構成を追加
            wallstruct = {'Name':prev_partname, \
                          'OutEmissiv':0.90, \
                          'OutSolarAbs':0.80, \
                          'InConHeatTrans':surfstrudct['hic'], \
                          'InRadHeatTrans':surfstrudct['hir'], \
                          'Layers':layers}
            Walls.append(wallstruct)
            #print(Walls)
            
            #壁体群の登録
            d = {
                'Common': {
                    'Region': 6,
                    'TimeInterval': 3600.,
                    'ResponseFactorCalculationHours': 50.
                },
                'Walls': Walls
            }
            #print('Dictionary 作成完了')
            #print(type(d))
            self.__wall_mng = WallMng(d)
            
            return self.__wall_mng
        

In [3]:
main = heat_load_main()
sch = main.Schedule_init()

In [4]:
wall_mng = main.wall_mng()
#print(wall_mng.RFT('外壁'))
#print(wall_mng.RFA('外壁'))
#print(wall_mng.Row('外壁'))

In [5]:
window_mng = main.window_mng()
#print(type(window_mng))
window_type = window_mng.Window('窓')
strWindowName = '窓2'
print('開口部の室内表面から屋外までの熱貫流率 :', window_mng.Window(strWindowName).Uso() )
print('日射透過率 :', window_mng.Window(strWindowName).T() )
print('吸収日射取得率 :', window_mng.Window(strWindowName).B() )
print('室外側熱伝達率 :', window_mng.Window(strWindowName).ho() )
print('室内側熱伝達率 :', window_mng.Window(strWindowName).hi() )
print('室内側対流熱伝達率 :', window_mng.Window(strWindowName).hic() )
print('室内側放射熱伝達率 :', window_mng.Window(strWindowName).hir() )
print('室外側放射率 :', window_mng.Window(strWindowName).Eo() )

開口部の室内表面から屋外までの熱貫流率 : 3.1319054652880354
日射透過率 : 0.6
吸収日射取得率 : 0.05
室外側熱伝達率 : 25.0
室内側熱伝達率 : 9.1
室内側対流熱伝達率 : 4.1
室内側放射熱伝達率 : 5.0
室外側放射率 : 0.9


In [6]:
objGdata = main.Gdata_init()
print(objGdata.ApDate())
print(objGdata.DetailOut())
print(objGdata.DTime())
print(objGdata.EnDate())
print(objGdata.FFcalcMethod())
print(objGdata.FlgOrig(datetime.date(1990,1,1)))
print(objGdata.FlgOrig(datetime.date(1889,12,31)))

1988-12-12
True
3600
1989-12-31
面積比
True
False


In [7]:
weatherdata = main.Weather_init()
wdt = weatherdata.WeaData(enmWeatherComponent.Ta,datetime.datetime(2017,1,2,0,30,0),True)
print(wdt)

0.5 5.9 5.4
5.65
